# Causal ML

Source: https://www.pywhy.org/dowhy/v0.11.1/example_notebooks/dowhy-conditional-treatment-effects.html

In [ ]:
# !pip install dowhy econml

In [ ]:
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets

import econml
import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 생성
실습을 위해 인위적인 데이터를 생성합니다. `dowhy.datasets.linear_dataset`를 사용하여 데이터를 생성하며, 인과 추정을 위한 다양한 변수를 포함합니다.

In [ ]:
BETA = 10

data = dowhy.datasets.linear_dataset(BETA, num_common_causes=4, num_samples=10000,
                                    num_instruments=2, num_effect_modifiers=2,
                                    num_treatments=1,
                                    treatment_is_binary=False,
                                    num_discrete_common_causes=2,
                                    num_discrete_effect_modifiers=0,
                                    one_hot_encode=False)
df = data['df']
print(df.head())
print("True causal estimate is", data["ate"])

## 2. 인과 모델 정의 및 시각화
생성된 데이터를 사용하여 인과 모델을 정의하고, DAG(Directed Acyclic Graph)를 시각화합니다.

In [ ]:
model = CausalModel(data=data["df"],
                    treatment=data["treatment_name"], outcome=data["outcome_name"],
                    graph=data["gml_graph"])

model.view_model()

## 3. DR Learner를 사용한 인과 효과 추정
이제 DR Learner를 사용하여 CATE를 추정합니다.
* Binary 데이터 생성: Binary 처리 변수를 포함하는 데이터를 생성합니다.
* 모델 정의 및 시각화: Binary 데이터를 사용하여 인과 모델을 정의하고, DAG를 시각화합니다.
* 인과 효과 식별: Binary 데이터에서 인과 효과를 식별합니다.
* DR Learner를 사용한 인과 효과 추정: econml 라이브러리를 사용하여 DR Learner를 구현하고, CATE를 추정합니다.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# Binary 데이터를 생성합니다.
data_binary = dowhy.datasets.linear_dataset(beta=BETA, num_common_causes=4, num_samples=10000,
                                            num_instruments=2, num_effect_modifiers=2,
                                            num_treatments=1, treatment_is_binary=True,
                                            num_discrete_common_causes=2,
                                            num_discrete_effect_modifiers=0,
                                            one_hot_encode=True)
df_binary = data_binary['df']

model_binary = CausalModel(data=data_binary["df"],
                           treatment=data_binary["treatment_name"], outcome=data_binary["outcome_name"],
                           graph=data_binary["gml_graph"])

model_binary.view_model()

identified_estimand_binary = model_binary.identify_effect(proceed_when_unidentifiable=True)

drlearner_estimate = model_binary.estimate_effect(
    identified_estimand_binary,
    method_name="backdoor.econml.dr.LinearDRLearner",
    confidence_intervals=False,
    method_params={"init_params":{
        'model_propensity': LogisticRegressionCV(cv=3, solver='lbfgs', multi_class='auto')
        },
        "fit_params":{}
        }
     )
print(drlearner_estimate)
print("True causal estimate is", data_binary["ate"])

1. Identified Estimand (식별된 추정량): 우리가 추정하려는 인과 효과입니다. 여러 변수를 조정하여 처리(treatment)가 결과에 미치는 영향을 평가합니다.
2. Realized Estimand (실현된 추정량): 실제로 사용된 모델입니다. 이 모델은 여러 변수를 고려하여 결과를 예측합니다.
3. Estimate (추정치): 처리(treatment)가 결과에 미치는 평균적인 효과입니다.
4. True causal estimate (실제 인과 추정 값): 시뮬레이션 데이터에서 실제로 설정한 인과 효과입니다.